In [6]:
import time
import random
import numpy as np
import pandas as pd



## Ejercicio 1

### Carga de ejemplares

In [3]:
# Los ejemplares se subieron a un repositorio de Github por lo que para descargarlos y poder usarlos se tiene que ejecutar la siguiente celda:
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/pr152.tsp
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/eil51.tsp
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/ch130.tsp
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/berlin52.tsp

--2024-09-19 13:23:47--  https://raw.githubusercontent.com/FridaVargas/tsp/main/pr152.tsp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2206 (2.2K) [text/plain]
Saving to: ‘pr152.tsp’

pr152.tsp           100%[===================>]   2.15K  --.-KB/s    in 0s      

2024-09-19 13:23:47 (26.2 MB/s) - ‘pr152.tsp’ saved [2206/2206]

--2024-09-19 13:23:47--  https://raw.githubusercontent.com/FridaVargas/tsp/main/eil51.tsp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 579 [text/plain]
Saving to: ‘eil51.tsp’

eil51.tsp           100%

### Implementa un algoritmo que lea un archivo de ejemplares para el TSP.

In [4]:
def lectura_TSP(ejemplar, imprimir = False):
  """
  Función que recibe de parámetro un ejemplar de TSP con el formato especificado en TSPLIB y de parámetro opcional si quieres imprimir en pantalla el nombre
  y la dimension del ejemplar, por default no se imprime.
  Devuelve un diccionario con sus atributos y una tabla
  con los nodos(ciudades) y las coordenadas de dónde están. Además imprime en pantalla el nombre y la dimensión del ejemplar.
  """
  archivo = open(ejemplar,"r")
  contenido = archivo.readlines()
  datos = {} # Aquí se van a guardar los atributos del ejemplar
  linea = 0 # Nos dice cuántas líneas son de especificaciones
  while ':' in contenido[linea]: # Se busca el caracter especial ":" para separar la palara clave del valor
    atributo = contenido[linea].replace('\n', "")
    llave, valor = atributo.split(':',1) # Se guarda en el diccionario
    llave = llave.replace(" ", "")
    datos[llave] = valor
    linea += 1
  datos['lectura'] = linea
  dimension = int(datos['DIMENSION'])
  ciudades = contenido[linea+1:]
  # Para ver la cantidad de coordenadas nos fijamos en el primer set de datos que es el nodo cor1, cor2..
  n = len(ciudades[0].split(" ")) # hay n-1 coordenadas
  listas = [[] for i in range(n)]# Creamos una lista de listas para ir almacenando las coordenadas y nodos para despues pasarlo a un DataFrame
  for i in range(0, dimension):
    ciudad = ciudades[i].replace('\n', "")
    partes = ciudad.split(" ")
    for j in range(0, n):
      listas[j].append(float(partes[j]))
  columnas = ["no. ciudad"] + ["coordenada" + str(i) for i in range(1,n)]
  # Creamos un dataframe
  coordenadas= pd.DataFrame(np.transpose(listas), columns= columnas, index = [i for i in range(1, dimension+1)])
  # Se imprimen en pantalla las caracteristicas del ejemplar si imprimir = True
  if imprimir == True:
    print("El nombre del ejemplar es: ", datos['NAME'], "\n")
    print("La dimensión del ejemplar es: ", datos['DIMENSION'], "\n")
    # Si se desean más especificaciones hay que descomentar las siguientes dos lineas paa que se impriman
    #print("Otras especificaciones: \n")
    #print(datos)
  return datos, coordenadas




Probamos en los 4 ejemplares la implementación:

In [7]:
lectura_TSP("berlin52.tsp", True)
print("------------------------------\n")
lectura_TSP("ch130.tsp", True)
print("------------------------------\n")
lectura_TSP("eil51.tsp", True)
print("------------------------------\n")
lectura_TSP("pr152.tsp", True)
print("------------------------------\n")

El nombre del ejemplar es:   berlin52 

La dimensión del ejemplar es:   52 

------------------------------

El nombre del ejemplar es:   ch130 

La dimensión del ejemplar es:   130 

------------------------------

El nombre del ejemplar es:   eil51 

La dimensión del ejemplar es:   51 

------------------------------

El nombre del ejemplar es:   pr152 

La dimensión del ejemplar es:   152 

------------------------------



### Ahora realizamos la matriz de distancias
Empleando el hecho que $d(i,j) = d(j,i)$ reducimos la complejidad computacional del problema. Pues $d(1,j) = d(j,1)$ y así nos queda una matriz de $(n-1) \cdot (n-1)$ luego, $d(2,j) = d(i,2)$ y así sucesivamente. Concluyéndose así que sólo es necesario un ciclo _for_.

In [8]:
def matriz_distancias(ejemplar):
  """
  Función que recibe de parámetro un ejemplar de TSP con el formato especificado en TSPLIB y devuelve su matriz de distancias asociada
  """
  datos, coordenadas = lectura_TSP(ejemplar)
  n = int(datos['DIMENSION'])
  matriz = np.zeros((n,n))
  # Calcula distancias entre cada par de nodos
  for i in range(n):  # Itera sobre cada nodo
      for j in range(i + 1, n):  # Solo itera sobre la mitad trangular superior derecha
          distancia = np.sqrt((coordenadas['coordenada1'].iloc[i] - coordenadas['coordenada1'].iloc[j])**2 +
                              (coordenadas['coordenada2'].iloc[i] - coordenadas['coordenada2'].iloc[j])**2)
          matriz[i][j] = distancia  # Almacena la distancia en la matriz
          matriz[j][i] = distancia  # La distancia es la misma de j a i

  return matriz


### Implementa un algoritmo para generar soluciones aleatorias para el TSP, utilizando algún esquema de codificación basado en permutaciones

In [9]:
# Usamos la funcion permutation de numpy, dejando fijo 1 y permutando de 2 a n
# Convertimos a lista después para un problema en particular, por lo que la llamaremos dentro de la función

def generar_sol_aleatoria(ejemplar, semilla =int(time.time())):
  """
  Función que a partir del archivo del ejemplar y una semilla, genera una solucion (permutacion) aleatoria al ejemplar
  """
  np.random.seed(semilla)
  datos, coordenadas = lectura_TSP(ejemplar)
  dimension = int(datos['DIMENSION'])
  solucion = list([1] + list(np.random.permutation(list(range(2, dimension + 1)))))
  return solucion # Como lista


In [ ]:
# Ejemplo de uso
#generar_sol_aleatoria('pr152.tsp')

### Implementa un algoritmo para evaluar una solución (permutación)

In [10]:
def evaluar_sol(ejemplar, solucion):
  """
  Dado un ejemplar y una solución, encuentra el costo de esa solucion a través de la matriz de distancias
  """
  matriz = matriz_distancias(ejemplar) # llamamos matriz de distancias
  # inicializamos la distancia total
  total = 0
  for i in range(len(solucion)-1):
    a = int(solucion[i])-1 #Hacemos que el index comience en cero
    b = int(solucion[i+1])-1 # Primero los pasamos a enteros para que los reconozca adecuadamente en la matriz
    total += int(matriz[a][b])
  # Falta calcular la distancia del fin de la lista a 1
  c = int(solucion[-1])-1
  total += int(matriz[c][1])

  return total

In [11]:
# Lo probamos para un ejemplar cualquiera de los dados:
solucion = generar_sol_aleatoria('ch130.tsp')
costo = evaluar_sol('ch130.tsp', solucion)
print("El costo de la solucion es", costo)


El costo de la solucion es 45958


### Programa 1
Programa que reciba como parámetros (en la misma línea de ejecución):

 * Nombre del archivo con los datos del ejemplar
 * Semilla del generador de aleatorio
 * Nombre del archivo con los datos de la solución generada

Como salida, deberá imprimir algunos datos generales del ejemplar leído. Por ejemplo:

  * Nombre del ejemplar (archivo)
  * Tamaño del ejemplar (número de ciudades)
  * Arista de mayor peso (o distancia más grande)
  * Ejemplo de solución (solo los primeras y los últimas 3 elementos de la permutación)
  * En caso de que se haya indicado, se deberá escribir en un archivo de texto la solución generada (completa)


In [12]:
def arista_mayor(ejemplar):
  """
  Recibe el archivo con el ejemplar y devuelve el peso de la arista con mayor peso y entre cuáles ciudades se encuentra
  """
  matriz = matriz_distancias(ejemplar)
  n = len(matriz)
  # Inicializamos variables para ahi guardar el no. de columna y no. de fila del maayor valor entre ciudades
  i_mayor = -1
  j_mayor = -1
  valor_mayor = -1

  for i in range(n):
    for j in range(n):
      if float(matriz[i][j]) > valor_mayor:
        i_mayor = i
        j_mayor = j
        valor_mayor = float(matriz[i][j])
  return i_mayor+1, j_mayor+1, valor_mayor # Sumamos 1 para que el index de las ciudades comience en 1 como en los originales


def lectura_ejemplar(ejemplar, semilla=int(time.time()), nombre_archivo=str(time.time())):
  """
  Parámetros y salida especificados al inicio de la sección
  """
  datos, coordenadas = lectura_TSP(ejemplar, True) # True para imprimir nombre y dimension
  i_mayor, j_mayor, valor_mayor = arista_mayor(ejemplar)
  # Imprimir arista de mayor valor y entre cuales ciudades está
  print("La arista de mayor peso se encuentra entre las ciudades ", i_mayor, "y ", j_mayor, "con un peso de ", valor_mayor,"\n")
  # Ejemplo de sol e imprimimos sus primeros y últimos 3 valores
  solucion = generar_sol_aleatoria(ejemplar, semilla)
  print("Una solución al ejemplar es:", solucion[0], solucion[1], solucion[2], ",...,", solucion[-3], solucion[-2], solucion[-1])

  archivo = nombre_archivo + ".txt"
  # Guardamos los datos de nombre, dimension como lo especifica TSPLIB y la solucion en el archivo
  contenido = (
    f"NAME : {datos['NAME']}\n"
    f"DIMENSION: {datos['DIMENSION']}\n"
    f"Una solución al ejemplar es \n"
  )
  contenido += ',\n '.join(str(i) for i in solucion)

  with open(nombre_archivo, 'w') as archivo:
    archivo.write(contenido)

  print('La solucion se guardó en el archivo ', f'{nombre_archivo}.txt')


In [13]:
# Lo probamos con ejemplar cualquiera de los dados
lectura_ejemplar('eil51.tsp', 123, 'archivo_sol')

El nombre del ejemplar es:   eil51 

La dimensión del ejemplar es:   51 

La arista de mayor peso se encuentra entre las ciudades  36 y  40 con un peso de  85.63293758829018 

Una solución al ejemplar es: 1 12 15 ,..., 30 4 47
La solucion se guardó en el archivo  archivo_sol.txt


## Programa 2
Programa reciba como parámetros (en la misma línea de ejecución) :

* Nombre del archivo con los datos del ejemplar
* Nombre del archivo con los datos de la solución a evaluar

Como salida, deberá imprimir algunos datos generales del ejemplar leído. Por ejemplo:

* Nombre del ejemplar (archivo)
* Tamaño del ejemplar (número de ciudades)
* Costo de la solución


In [14]:
def leer_sol(archivo_sol, imprimir = False):
  # Abrimos y leemos el archivo, guardando los datos como en la funcion leer_archivo
  with open(archivo_sol, 'r') as f:
    contenido = f.readlines()
  datos = {} # Aquí se van a guardar los atributos del ejemplar
  linea = 0 # Nos dice cuántas líneas son de especificaciones
  solucion = []
  while ':' in contenido[linea]: # Se busca el caracter especial ":" para separar la palara clave del valor
    atributo = contenido[linea].replace('\n', "")
    llave, valor = atributo.split(':',1) # Se guarda en el diccionario
    llave = llave.replace(" ", "")
    datos[llave] = valor
    linea += 1
  datos['lectura'] = linea
  dimension = int(datos['DIMENSION'])
  ciudades = contenido[linea+1:]
  # Nos deshacemos de salto de linea con strip( ) y de la coma con rstrip()
  solucion = [int(ciudad.strip().rstrip(',')) for ciudad in ciudades]
  if imprimir == True:
    print("Ejemplar: ", datos['NAME'], "\n")
    print("Dimension: ", datos['DIMENSION'], "\n")
    print("Solucion en archivo: ", solucion[0], solucion[1], solucion[2], ",...,", solucion[-3], solucion[-2], solucion[-1])
  return solucion


def lectura_solucion(ejemplar, archivo_sol):
  datos, coordenadas = lectura_TSP(ejemplar, True) # True para imprimir nombre y dimension
  solucion = leer_sol(archivo_sol)
  costo = evaluar_sol('ch130.tsp', solucion)
  print("El costo de la solucion es", costo)




In [15]:
lectura_solucion('ch130.tsp', 'archivo_sol')

El nombre del ejemplar es:   ch130 

La dimensión del ejemplar es:   130 

El costo de la solucion es 15271
